In [35]:
import geopandas as gpd
import pandas as pd
import numpy as np

## Configuraciones Iniciales
estadoCivil_distrito_2011_csv_path = r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\05 - Estado Civil\Distritos\Ocupados de 16 y más años_EstadoCivil_Distritos_Censo_2011.csv"
estadoCivil_distrito_2021_csv_path = r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\05 - Estado Civil\Distritos\Ocupados de 16 y más años_EstadoCivil_Distritos_Censo_2021.csv"
cartografia_distrito_shp_path = r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Shapefiles\da_cartografiaDistritoMunicipal-25830\da_cartografiaDistritoMunicipalPolygon.shp"

## Lectura inicial de datos estado civil
df_estadoCivil_distrito_2011 =  pd.read_csv(estadoCivil_distrito_2011_csv_path,sep=";")
df_estadoCivil_distrito_2021 =  pd.read_csv(estadoCivil_distrito_2021_csv_path,sep=";")
df_cartografia_distrito = gpd.read_file(cartografia_distrito_shp_path)

# Ver las columnas
#print(df_estadoCivil_distrito_2011.columns)
#print(df_estadoCivil_distrito_2021.columns)

# Inspeccionar los primeros registros
#print(df_estadoCivil_distrito_2011.head())
#print(df_estadoCivil_distrito_2021.head())

#Cambio el nombre de la columna 'Distrito ' a Distrito de residencia
df_estadoCivil_distrito_2021.rename(columns={'Distrito ': 'Distrito de residencia'}, inplace=True)

#Se realiza una normalizacion de los csv de ocupacion en dos columnas ocupacion y personas 
df_estadoCivil_distrito_2011_long = pd.melt(df_estadoCivil_distrito_2011, id_vars=["Municipio de residencia","Distrito de residencia"], var_name='Estado Civil', value_name='Personas')
#print(df_estadoCivil_distrito_2011_long.columns)
#print(df_estadoCivil_distrito_2011_long.head())
df_estadoCivil_distrito_2021_long = pd.melt(df_estadoCivil_distrito_2021, id_vars=["Municipio de residencia","Distrito de residencia"], var_name='Estado Civil', value_name='Personas')
#print(df_estadoCivil_distrito_2021_long.columns)
#print(df_estadoCivil_distrito_2021_long.head())

#Contamos el numero de valores que hay en estos casos
num_filas_en_blanco_2011 = (df_estadoCivil_distrito_2011_long['Personas'].isna()).sum()
num_filas_con_asterisco_2011 = (df_estadoCivil_distrito_2011_long['Personas'] == "*").sum()
num_filas_contiene_asterisco_2011 = (df_estadoCivil_distrito_2011_long['Personas'].apply(str).str.contains(r'\*')).sum()
#print(num_filas_en_blanco_2011)#0
#print(num_filas_con_asterisco_2011)#0
#print(num_filas_contiene_asterisco_2011)#6
num_filas_en_blanco_2021 = (df_estadoCivil_distrito_2021_long['Personas'].isna()).sum()
num_filas_con_asterisco_2021 = (df_estadoCivil_distrito_2021_long['Personas'] == "*").sum()
num_filas_contiene_asterisco_2021 = (df_estadoCivil_distrito_2021_long['Personas'].apply(str).str.contains(r'\*')).sum()
#print(num_filas_en_blanco_2021)#0
#print(num_filas_con_asterisco_2021)#0
#print(num_filas_contiene_asterisco_2021)#0

## Eliminar filas ##
#Eliminar filas de municipio Total
df_estadoCivil_distrito_2011_long = df_estadoCivil_distrito_2011_long[df_estadoCivil_distrito_2011_long['Municipio de residencia'] != 'Total']
df_estadoCivil_distrito_2021_long = df_estadoCivil_distrito_2021_long[df_estadoCivil_distrito_2021_long['Municipio de residencia'] != 'Total']
#Eliminar filas de Distrito de residencia Total
df_estadoCivil_distrito_2011_long = df_estadoCivil_distrito_2011_long[df_estadoCivil_distrito_2011_long['Distrito de residencia'] != 'Total']
df_estadoCivil_distrito_2021_long = df_estadoCivil_distrito_2021_long[df_estadoCivil_distrito_2021_long['Distrito de residencia'] != 'Total']
#Eliminamos las filas de Estado Civil Total y No consta
#print(df_estadoCivil_distrito_2011_long['Estado Civil'].unique())
#print(df_estadoCivil_distrito_2021_long['Estado Civil'].unique())
df_estadoCivil_distrito_2011_long = df_estadoCivil_distrito_2011_long[df_estadoCivil_distrito_2011_long['Estado Civil'] != 'Total']
df_estadoCivil_distrito_2021_long = df_estadoCivil_distrito_2021_long[df_estadoCivil_distrito_2021_long['Estado Civil'] != 'Total']
df_estadoCivil_distrito_2011_long = df_estadoCivil_distrito_2011_long[df_estadoCivil_distrito_2011_long['Estado Civil'] != 'No consta']
df_estadoCivil_distrito_2021_long = df_estadoCivil_distrito_2021_long[df_estadoCivil_distrito_2021_long['Estado Civil'] != 'No consta']

#print(df_estadoCivil_distrito_2011_long.head())
#print(df_estadoCivil_distrito_2021_long.head())

## Añadir columnas ##
#Añadir columna NUMERO con el numero de distrito
df_estadoCivil_distrito_2011_long["NUMERO"] = df_estadoCivil_distrito_2011_long["Distrito de residencia"].str[-2:].astype(float)
df_estadoCivil_distrito_2021_long["NUMERO"] = df_estadoCivil_distrito_2021_long["Distrito de residencia"].str[-2:].astype(float)

#print(df_estadoCivil_distrito_2011_long.head())
#print(df_estadoCivil_distrito_2021_long.head())

## Gestion de Nulos y valores extraños ##
# Limpiar los datos: eliminar el asterisco al final, pero dejar intacto el '*'
df_estadoCivil_distrito_2011_long['Personas'] = df_estadoCivil_distrito_2011_long['Personas'].replace(r'(\d+)\*$', r'\1', regex=True)
df_estadoCivil_distrito_2021_long['Personas'] = df_estadoCivil_distrito_2021_long['Personas'].replace(r'(\d+)\*$', r'\1', regex=True)

#Contamos el numero de valores que hay en estos casos
num_filas_en_blanco_2011 = (df_estadoCivil_distrito_2011_long['Personas'].isna()).sum()
num_filas_con_asterisco_2011 = (df_estadoCivil_distrito_2011_long['Personas'] == "*").sum()
num_filas_contiene_asterisco_2011 = (df_estadoCivil_distrito_2011_long['Personas'].apply(str).str.contains(r'\*')).sum()
#print(num_filas_en_blanco_2011)#0
#print(num_filas_con_asterisco_2011)#0
#print(num_filas_contiene_asterisco_2011)#0
num_filas_en_blanco_2021 = (df_estadoCivil_distrito_2021_long['Personas'].isna()).sum()
num_filas_con_asterisco_2021 = (df_estadoCivil_distrito_2021_long['Personas'] == "*").sum()
num_filas_contiene_asterisco_2021 = (df_estadoCivil_distrito_2021_long['Personas'].apply(str).str.contains(r'\*')).sum()
#print(num_filas_en_blanco_2021)#0
#print(num_filas_con_asterisco_2021)#0
#print(num_filas_contiene_asterisco_2021)#0

#Columna Personas como float
df_estadoCivil_distrito_2011_long['Personas'] = df_estadoCivil_distrito_2011_long['Personas'].astype(float)
df_estadoCivil_distrito_2021_long['Personas'] = df_estadoCivil_distrito_2021_long['Personas'].astype(float)


## Unificar valores categorias Estado Civil en censos 2011 y 2021
#Los valores que toma el campo Estado Civil son distintos
#print(df_estadoCivil_distrito_2011_long['Estado Civil'].unique())#Tamaño 5 ['Soltero/a' 'Casado/a' 'Separado/a' 'Divorciado/a' 'Viudo/a']
#print(df_estadoCivil_distrito_2021_long['Estado Civil'].unique())#Tamaño 4 ['Soltero/a' 'Casado/a' 'Viudo/a' 'Divorciado/a o Separado/a']

#print(df_estadoCivil_distrito_2011_long[df_estadoCivil_distrito_2011_long["Estado Civil"] == "Divorciado/a"])
#print(df_estadoCivil_distrito_2011_long[df_estadoCivil_distrito_2011_long["Estado Civil"] == "Separado/a"])

#En 2011 sumamos las categorias Separado y Divorciado en una misma como en 2021 para que sean comparables
# Crear una nueva categoría "divorciados y separados"
def group_divorced_separated(df):
    # Filtrar por los registros de divorciados y separados
    filtered = df[df["Estado Civil"].isin(["Separado/a", "Divorciado/a"])]
    
    # Agrupar y sumar las personas por Municipio y Distrito
    grouped = filtered.groupby(["Municipio de residencia", "Distrito de residencia","NUMERO"], as_index=False)["Personas"].sum()
    grouped["Estado Civil"] = "Divorciado/a o Separado/a"

    return grouped

# Obtener los datos agrupados
grouped_data = group_divorced_separated(df_estadoCivil_distrito_2011_long)

# Eliminar las filas originales de divorciados y separados y añadir la nueva categoría
df_estadoCivil_distrito_2011_long = pd.concat([
    df_estadoCivil_distrito_2011_long[~df_estadoCivil_distrito_2011_long["Estado Civil"].isin(["Separado/a", "Divorciado/a"])],
    grouped_data
], ignore_index=True)

# Ordenar los datos para facilitar la lectura
#final_df = final_df.sort_values(by=["Municipio de residencia", "Distrito de residencia", "Estado Civil"])

#print(df_estadoCivil_distrito_2011_long[df_estadoCivil_distrito_2011_long["Estado Civil"] == "Divorciado/a o Separado/a"])

#print(df_estadoCivil_distrito_2011_long.count())#44 registros
#print(df_estadoCivil_distrito_2021_long.count())#44 registros
#print(df_estadoCivil_distrito_2011_long.head())
#print(df_estadoCivil_distrito_2021_long.head())

## Pasar los valores a relativo (Porcentajes) ##
#Calcular el total de personas por distrito
df_estadoCivil_distrito_2011_long['Total_personas_distrito'] = df_estadoCivil_distrito_2011_long.groupby('Distrito de residencia')['Personas'].transform('sum')
df_estadoCivil_distrito_2021_long['Total_personas_distrito'] = df_estadoCivil_distrito_2021_long.groupby('Distrito de residencia')['Personas'].transform('sum')
#Calcular porcentaje sobre el total
df_estadoCivil_distrito_2011_long['Personas %'] = (df_estadoCivil_distrito_2011_long['Personas'] / df_estadoCivil_distrito_2011_long['Total_personas_distrito'])
df_estadoCivil_distrito_2021_long['Personas %'] = (df_estadoCivil_distrito_2021_long['Personas'] / df_estadoCivil_distrito_2021_long['Total_personas_distrito'])

#print(df_estadoCivil_distrito_2011_long[df_estadoCivil_distrito_2011_long["NUMERO"] == 2])
#print(df_estadoCivil_distrito_2021_long[df_estadoCivil_distrito_2021_long["NUMERO"] == 2])

#Pivotar la tabla de nuevo para crear 4 variables, una por categoria de Estado Civil con el % ; una fila por distrito
df_estadoCivil_distrito_2011_long  = df_estadoCivil_distrito_2011_long.pivot(index='NUMERO', columns='Estado Civil', values='Personas %')
df_estadoCivil_distrito_2021_long  = df_estadoCivil_distrito_2021_long.pivot(index='NUMERO', columns='Estado Civil', values='Personas %')

print(df_estadoCivil_distrito_2011_long)
print(df_estadoCivil_distrito_2021_long)

##Export 4 variables
df_estadoCivil_distrito_2011_long.to_csv(r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\05 - Estado Civil\Distritos\2011_Distritos_EstadoCivil.csv",index=True,sep=";")
df_estadoCivil_distrito_2021_long.to_csv(r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\05 - Estado Civil\Distritos\2021_Distritos_EstadoCivil.csv",index=True,sep=";")


Estado Civil  Casado/a  Divorciado/a o Separado/a  Soltero/a   Viudo/a
NUMERO                                                                
1.0           0.464567                   0.083990   0.446194  0.005249
2.0           0.628515                   0.077228   0.282970  0.011287
3.0           0.339286                   0.032143   0.610714  0.017857
4.0           0.470956                   0.102770   0.414656  0.011618
5.0           0.301205                   0.068273   0.594378  0.036145
6.0           0.531119                   0.079759   0.378138  0.010983
7.0           0.554634                   0.066934   0.366562  0.011871
8.0           0.623906                   0.057840   0.310825  0.007429
9.0           0.487364                   0.078733   0.416213  0.017691
10.0          0.577072                   0.076820   0.334510  0.011599
11.0          0.621021                   0.055514   0.319763  0.003701
Estado Civil  Casado/a  Divorciado/a o Separado/a  Soltero/a   Viudo/a
NUMERO